# Normal - Compare coefficient estimates for IHT and lasso

In [20]:
using MendelIHT
using MendelGWAS
using SnpArrays
using DataFrames
using Distributions
using BenchmarkTools
using Random
using LinearAlgebra
using CSV
using GLM
using RCall
R"library(glmnet)"

RObject{StrSxp}
 [1] "glmnet"    "foreach"   "Matrix"    "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     


In [28]:
#run n repeats of L0_reg using the same X and b, but different y
function run_repeat(
    n        :: Int,
    p        :: Int,
    repeats  :: Int, 
    z        :: AbstractMatrix,
    true_b   :: Vector,
    cor_pos  :: Vector, #correct position of the true model
    d        :: UnionAll,
    l        :: GLM.Link
    )
    
    k = size(cor_pos, 1)
    IHT_estimated_β = zeros(k, repeats)
    lasso_estimated_β = zeros(k, repeats)
    marginal_estimated_β = zeros(k, repeats)
    
    for i in 1:repeats
        # simulat SNP data
        x = simulate_random_snparray(n, p, "tmp.bed")
        xbm = SnpBitMatrix{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true); 
        x_float = [convert(Matrix{Float64}, x, center=true, scale=true) z] #Float64 version of x

        #simulate phenotypes (e.g. vector y)
        if d == Normal || d == Poisson || d == Bernoulli
            prob = GLM.linkinv.(l, xbm * true_b)
            clamp!(prob, -20, 20)
            y = [rand(d(i)) for i in prob]
        elseif d == NegativeBinomial
            r = 10
            μ = GLM.linkinv.(l, xbm * true_b)
            prob = 1 ./ (1 .+ μ ./ r)
            y = [rand(d(r, i)) for i in prob] #number of failtures before r success occurs
        elseif d == Gamma
            μ = GLM.linkinv.(l, xbm * true_b)
            β = 1 ./ μ # here β is the rate parameter for gamma distribution
            y = [rand(d(α, i)) for i in β] # α is the shape parameter for gamma
        end
        y = Float64.(y)

        #compute results for data
        IHT_result = L0_reg(x, xbm, z, y, 1, k, d(), l)

        #compute lasso results. choose λ via cross validation
        num_folds = 3
        folds = rand(1:num_folds, size(x, 1))
        @rput x_float y folds num_folds #make variables visible to R
        R"lasso_cv_result = cv.glmnet(x_float, y, nfolds = num_folds, foldid = folds, family='gaussian')"
        R"lasso_beta_tmp = glmnet(x_float, y, lambda=lasso_cv_result$lambda.min, family='gaussian')$beta"
        R"lasso_beta = as.vector(lasso_beta_tmp)"
        @rget lasso_cv_result lasso_beta #pull result from R to Julia

        # Now run MendelGWAS (marginal analysis)
        make_bim_fam_files(x, y, "tmp") #create .bim and .bed files for MendelGWAS
        open("tmp_control.txt", "w") do f
            write(f, "plink_input_basename = tmp \n")
            write(f, "plink_field_separator = '\t' \n")
            write(f, "output_table = tmp_table.txt \n")
            write(f, "regression = linear \n")
            write(f, "regression_formula = Trait ~ 1\n")
        end
        GWAS("tmp_control.txt")
        result = CSV.read("tmp_table.txt")
        
        #store the correct position in estimated model
        IHT_estimated_β[:, i] .= IHT_result.beta[cor_pos]
        lasso_estimated_β[:, i] .= lasso_beta[cor_pos]
        marginal_estimated_β[:, i] .= result[cor_pos, 6]
    end
    
    return IHT_estimated_β, lasso_estimated_β, 
end

run_repeat (generic function with 1 method)

## Run 100 models

Here $\beta_{true}$ is the same in all models, but the design matrix and phenotype will be different. To run lasso, we first cross validate for the tuning parameter $\lambda$ before running the standard lasso. We compute the average and standard error for correctly identified predictors (i.e. estimated $\beta_i > 0$). Note that here we are not comparing false positives or false negatives. 

In [31]:
repeats = 100
n = 5000
p = 10000
d = Normal
l = canonicallink(d())

# set random seed for reproducibility
Random.seed!(2019)

# intercept
z = ones(Float64, n, 1)

#construct true model b
true_b = zeros(p)
true_b[1:6] = [0.01; 0.03; 0.05; 0.1; 0.25; 0.5]
shuffle!(true_b)
correct_position = findall(x -> x != 0, true_b)

#run repeats
@time iht_result, lasso_result, marginal_result = run_repeat(n, p, repeats, z, true_b, correct_position, d, l)

 
 
     Welcome to OpenMendel's
      GWAS analysis option
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/dev/MendelIHT/figures/repeats".

Keywords modified by the user:

  affected_designator = 2
  control_file = tmp_control.txt
  output_table = tmp_table.txt
  pedigree_file = tmp.fam
  plink_field_separator = 	
  plink_input_basename = tmp
  regression = linear
  regression_formula = Trait ~ 1
  snpdata_file = tmp.bed
  snpdefinition_file = tmp.bim
 


┌ Warning: `allowmissing` is a deprecated keyword argument
└ @ CSV /Users/biona001/.julia/packages/CSV/YgNdL/src/CSV.jl:263
┌ Warning: `allowmissing` is a deprecated keyword argument
└ @ CSV /Users/biona001/.julia/packages/CSV/YgNdL/src/CSV.jl:263


 
Analyzing the data.

 
Table of p-values for all SNPs is in file: tmp_table.txt
 
 
Mendel's analysis is finished.

 
 
     Welcome to OpenMendel's
      GWAS analysis option
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/dev/MendelIHT/figures/repeats".

Keywords modified by the user:

  affected_designator = 2
  control_file = tmp_control.txt
  output_table = tmp_table.txt
  pedigree_file = tmp.fam
  plink_field_separator = 	
  plink_input_basename = tmp
  regression = linear
  regression_formula = Trait ~ 1
  snpdata_file = tmp.bed
  snpdefinition_file = tmp.bim
 


┌ Warning: `allowmissing` is a deprecated keyword argument
└ @ CSV /Users/biona001/.julia/packages/CSV/YgNdL/src/CSV.jl:263
┌ Warning: `allowmissing` is a deprecated keyword argument
└ @ CSV /Users/biona001/.julia/packages/CSV/YgNdL/src/CSV.jl:263


 
Analyzing the data.

 
Table of p-values for all SNPs is in file: tmp_table.txt
 
 
Mendel's analysis is finished.

 
 
     Welcome to OpenMendel's
      GWAS analysis option
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/dev/MendelIHT/figures/repeats".

Keywords modified by the user:

  affected_designator = 2
  control_file = tmp_control.txt
  output_table = tmp_table.txt
  pedigree_file = tmp.fam
  plink_field_separator = 	
  plink_input_basename = tmp
  regression = linear
  regression_formula = Trait ~ 1
  snpdata_file = tmp.bed
  snpdefinition_file = tmp.bim
 


...

┌ Warning: `allowmissing` is a deprecated keyword argument
└ @ CSV /Users/biona001/.julia/packages/CSV/YgNdL/src/CSV.jl:263
┌ Warning: `allowmissing` is a deprecated keyword argument
└ @ CSV /Users/biona001/.julia/packages/CSV/YgNdL/src/CSV.jl:263


 
Analyzing the data.

 
Table of p-values for all SNPs is in file: tmp_table.txt
 
 
Mendel's analysis is finished.

 
 
     Welcome to OpenMendel's
      GWAS analysis option
 
 
Reading the data.

The current working directory is "/Users/biona001/.julia/dev/MendelIHT/figures/repeats".

Keywords modified by the user:

  affected_designator = 2
  control_file = tmp_control.txt
  output_table = tmp_table.txt
  pedigree_file = tmp.fam
  plink_field_separator = 	
  plink_input_basename = tmp
  regression = linear
  regression_formula = Trait ~ 1
  snpdata_file = tmp.bed
  snpdefinition_file = tmp.bim
 


┌ Warning: `allowmissing` is a deprecated keyword argument
└ @ CSV /Users/biona001/.julia/packages/CSV/YgNdL/src/CSV.jl:263
┌ Warning: `allowmissing` is a deprecated keyword argument
└ @ CSV /Users/biona001/.julia/packages/CSV/YgNdL/src/CSV.jl:263


 
Analyzing the data.

 
Table of p-values for all SNPs is in file: tmp_table.txt
 
 
Mendel's analysis is finished.

3422.145256 seconds (284.32 M allocations: 289.927 GiB, 2.55% gc time)


([0.0 0.0 … 0.0 0.0; 0.0 0.059477155281053906 … 0.059477155281053906 0.059477155281053906; … ; 0.0 0.0 … 0.0 0.0; 0.519906292860183 0.48814000528764634 … 0.48814000528764634 0.48814000528764634], [0.0 0.0 … 0.0 0.0; 0.0 0.01177465486212541 … 0.01177465486212541 0.01177465486212541; … ; 0.0 0.0 … 0.0 0.0; 0.46627760380563504 0.43974746763673545 … 0.43974746763673545 0.43974746763673545], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 1.0576655930896044 1.3558253446255752 … 1.3558253446255752 1.3558253446255752])

In [33]:
iht_result

6×100 Array{Float64,2}:
 0.0       0.0        0.0        …  0.0        0.0        0.0      
 0.0       0.0594772  0.0594772     0.0594772  0.0594772  0.0594772
 0.269344  0.234147   0.234147      0.234147   0.234147   0.234147 
 0.111284  0.0827714  0.0827714     0.0827714  0.0827714  0.0827714
 0.0       0.0        0.0           0.0        0.0        0.0      
 0.519906  0.48814    0.48814    …  0.48814    0.48814    0.48814  

In [34]:
lasso_result

6×100 Array{Float64,2}:
 0.0        0.0        0.0        …  0.0        0.0        0.0      
 0.0        0.0117747  0.0117747     0.0117747  0.0117747  0.0117747
 0.218311   0.18559    0.18559       0.18559    0.18559    0.18559  
 0.0597749  0.0334391  0.0334391     0.0334391  0.0334391  0.0334391
 0.0        0.0        0.0           0.0        0.0        0.0      
 0.466278   0.439747   0.439747   …  0.439747   0.439747   0.439747 

In [32]:
marginal_result

6×100 Array{Float64,2}:
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.487961  0.432611  0.432611  0.432611     0.432611  0.432611  0.432611
 0.183967  0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 1.05767   1.35583   1.35583   1.35583   …  1.35583   1.35583   1.35583 

In [35]:
iht_non_zero = [iht_result[i, findall(!iszero, iht_result[i, :])] for i in 1:6]
lasso_non_zero = [lasso_result[i, findall(!iszero, lasso_result[i, :])] for i in 1:6]
marginal_non_zero = [marginal_result[i, findall(!iszero, marginal_result[i, :])] for i in 1:6];

In [36]:
iht_non_zero_mean = mean.(iht_non_zero)

6-element Array{Float64,1}:
 NaN                   
   0.059477155281053906
   0.23449880574182613 
   0.08305656890596311 
 NaN                   
   0.48845766816337177 

In [37]:
lasso_non_zero_mean = mean.(lasso_non_zero)

6-element Array{Float64,1}:
 NaN                   
   0.011774654862125412
   0.1859170832772628  
   0.033702455879150835
 NaN                   
   0.4400127689984244  

In [38]:
marginal_non_zero_mean = mean.(marginal_non_zero)

6-element Array{Float64,1}:
 NaN                  
 NaN                  
   0.4331642070058298 
   0.18396671550684718
 NaN                  
   1.3528437471102155 

In [41]:
iht_non_zero_std = std.(iht_non_zero)

6-element Array{Float64,1}:
 NaN                    
   0.0                  
   0.003519735398062406 
   0.0028512089923308256
 NaN                    
   0.003176628757253662 

In [42]:
lasso_non_zero_std = std.(lasso_non_zero)

6-element Array{Float64,1}:
 NaN                     
   1.7435516422837063e-18
   0.003272100452159023  
   0.0026335842355116623 
 NaN                     
   0.0026530136168899598 

In [39]:
marginal_non_zero_std = std.(marginal_non_zero)

6-element Array{Float64,1}:
 NaN                    
 NaN                    
   0.0055350098707253005
 NaN                    
 NaN                    
   0.02981597515359708  

In [43]:
final = DataFrame(
    true_β = true_b[correct_position],
    β_IHT_mean = iht_non_zero_mean, 
    β_IHT_std = iht_non_zero_std, 
    β_lasso_mean = lasso_non_zero_mean,
    β_lasso_std = lasso_non_zero_std,
    β_marginal_mean = marginal_non_zero_mean,
    β_marginal_std = marginal_non_zero_std)

,true_β,β_IHT_mean,β_IHT_std,β_lasso_mean,β_lasso_std,β_marginal_mean,β_marginal_std
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.03,NaN,NaN,NaN,NaN,NaN,NaN
2,0.05,0.0594772,0.0,0.0117747,1.74355e-18,NaN,NaN
3,0.25,0.234499,0.00351974,0.185917,0.0032721,0.433164,0.00553501
4,0.1,0.0830566,0.00285121,0.0337025,0.00263358,0.183967,NaN
5,0.01,NaN,NaN,NaN,NaN,NaN,NaN
6,0.5,0.488458,0.00317663,0.440013,0.00265301,1.35284,0.029816
